# Harmony EOSS API Tutorial

## Before you start
Before you beginning this tutorial, make sure you have an account in the Earthdata Login UAT environment, which 
will be used for this notebook by visiting [https://uat.urs.earthdata.nasa.gov](https://uat.urs.earthdata.nasa.gov).
These accounts, as all Earthdata Login accounts, are free to create and only take a moment to set up.

## Set Up Authentication

We need some boilerplate up front to log in to Earthdata Login.  The function below will allow Python
scripts to log into any Earthdata Login application programmatically.  To avoid being prompted for
credentials every time you run and also allow clients such as curl to log in, you can add the following
to a `.netrc` (`_netrc` on Windows) file in your home directory:

```
machine uat.urs.earthdata.nasa.gov
    login <your username>
    password <your password>
```

Make sure that this file is only readable by the current user or you will receive an error stating
"netrc access too permissive."

`$ chmod 0600 ~/.netrc` 


In [9]:
from urllib import request
from http.cookiejar import CookieJar
import getpass
import netrc

def setup_earthdata_login_auth(endpoint):
    """
    Set up the request library so that it authenticates against the given Earthdata Login
    endpoint and is able to track cookies between requests.  This looks in the .netrc file 
    first and if no credentials are found, it prompts for them.

    Valid endpoints include:
        uat.urs.earthdata.nasa.gov - Earthdata Login UAT (Harmony's current default)
        urs.earthdata.nasa.gov - Earthdata Login production
    """
    try:
        username, _, password = netrc.netrc().authenticators(endpoint)
    except (FileNotFoundError, TypeError):
        # FileNotFound = There's no .netrc file
        # TypeError = The endpoint isn't in the netrc file, causing the above to try unpacking None
        print('Please provide your Earthdata Login credentials to allow data access')
        print('Your credentials will only be passed to %s and will not be exposed in Jupyter' % (endpoint))
        username = input('Username:')
        password = getpass.getpass()

    manager = request.HTTPPasswordMgrWithDefaultRealm()
    manager.add_password(None, endpoint, username, password)
    auth = request.HTTPBasicAuthHandler(manager)

    jar = CookieJar()
    processor = request.HTTPCookieProcessor(jar)
    opener = request.build_opener(auth, processor)
    request.install_opener(opener)


Now call the above function to set up Earthdata Login for subsequent requests

In [10]:
setup_earthdata_login_auth('uat.urs.earthdata.nasa.gov')

Please provide your Earthdata Login credentials to allow data access
Your credentials will only be passed to uat.urs.earthdata.nasa.gov and will not be exposed in Jupyter


Username: mgangl
 ·········


## Build the EOSS Root URL

Next we will build a URL for the EOSS service for a given granule.  To get data using the service, you need 
a [CMR UAT](https://cmr.uat.earthdata.nasa.gov) collection ID for a supported collection and the ID of a
granule within that collection.

By convention, all Harmony services are accessed through `<harmony_root>/<collection_id>/<service_name>`

In [11]:
harmony_root = 'https://harmony.uat.earthdata.nasa.gov'
config = {
    'collection_id': 'C1225996408-POCUMULUS',
    'eoss_version': '0.1.0'
}
eoss_collection_root = harmony_root+'/{collection_id}/eoss/{eoss_version}/items/'.format(**config)
print(eoss_collection_root)

https://harmony.uat.earthdata.nasa.gov/C1225996408-POCUMULUS/eoss/0.1.0/items/


### Find a granule for subsetting

We will use the CMR API to find information about our collections we will work on.


In [28]:
import json
import pprint

with request.urlopen('https://cmr.uat.earthdata.nasa.gov:443/search/concepts/C1225996408-POCUMULUS/1') as response:
    data = response.read()
    encoding = response.info().get_content_charset('utf-8')
    JSON_object = json.loads(data.decode(encoding))
    pp = pprint.PrettyPrinter(indent=4)
    pp.pprint(JSON_object)


{   'Abstract': 'The Advanced Microwave Scanning Radiometer 2 (AMSR2) was '
                'launched on 18 May 2012, onboard the Global Change '
                'Observation Mission - Water (GCOM-W) satellite developed by '
                'the Japan Aerospace Exploration Agency (JAXA). The GCOM-W '
                'mission aims to establish the global and long-term '
                'observation system to collect data, which is needed to '
                'understand mechanisms of climate and water cycle variations, '
                'and demonstrate its utilization. AMSR2 onboard the first '
                'generation of the GCOM-W satellite will continue Aqua/AMSR-E '
                'observations of water vapor, cloud liquid water, '
                'precipitation, SST, sea surface wind speed, sea ice '
                'concentration, snow depth, and soil moisture. AMSR2 is a '
                'remote sensing instrument for measuring weak microwave '
                'emission fro

### Find a granule from the collection

Using the collection concept, we can drill down and find granules of that collection. Note the 'granules.umm_json'. This is used because the default resposne (search/granules?) will return XML.

In [32]:
#https://cmr.uat.earthdata.nasa.gov/search/granules?collection_concept_id=C1225996408-POCUMULUS
with request.urlopen('https://cmr.uat.earthdata.nasa.gov/search/granules.umm_json?collection_concept_id=C1225996408-POCUMULUS') as response:
    data = response.read()
    encoding = response.info().get_content_charset('utf-8')
    JSON_object = json.loads(data.decode(encoding))
    pp = pprint.PrettyPrinter(indent=2)
    pp.pprint(JSON_object)

{ 'hits': 10239,
  'items': [ { 'meta': { 'concept-id': 'G1226018937-POCUMULUS',
                         'concept-type': 'granule',
                         'format': 'application/vnd.nasa.cmr.umm+json',
                         'native-id': '20171231231928-REMSS-L2P_GHRSST-SSTsubskin-AMSR2-L2B_rt_r29917-v02.0-fv01.0.nc.cmr.json',
                         'provider-id': 'POCUMULUS',
                         'revision-date': '2019-03-08T18:09:35Z',
                         'revision-id': 1},
               'umm': { 'CollectionReference': { 'ShortName': 'AMSR2-REMSS-L2P-v8a',
                                                 'Version': '1'},
                        'DataGranule': { 'ArchiveAndDistributionInformation': [ { 'Name': 'Not '
                                                                                          'provided',
                                                                                  'Size': 10242048,
                                                     

### Choose a granule and get some information

We will choose a random granuke (that just happens to work well with our services): 

```
<reference>
  <name>
    20180101041600-REMSS-L2P_GHRSST-SSTsubskin-AMSR2-L2B_rt_r29920-v02.0-fv01.0.nc
  </name>
  <id>G1226018995-POCUMULUS</id>
  <location>
    https://cmr.uat.earthdata.nasa.gov:443/search/concepts/G1226018995-POCUMULUS/1
  </location>
  <revision-id>1</revision-id>
</reference>
```

Again, we'll use CMR to find out more information about this granule:

In [30]:
with request.urlopen('https://cmr.uat.earthdata.nasa.gov/search/concepts/G1226018995-POCUMULUS/1') as response:
    data = response.read()
    encoding = response.info().get_content_charset('utf-8')
    JSON_object = json.loads(data.decode(encoding))
    pp = pprint.PrettyPrinter(indent=2)
    pp.pprint(JSON_object)

{ 'CollectionReference': {'ShortName': 'AMSR2-REMSS-L2P-v8a', 'Version': '1'},
  'DataGranule': { 'ArchiveAndDistributionInformation': [ { 'Name': '20180101041600-REMSS-L2P_GHRSST-SSTsubskin-AMSR2-L2B_rt_r29920-v02.0-fv01.0.nc',
                                                            'Size': 10242048,
                                                            'SizeUnit': 'NA'}],
                   'DayNightFlag': 'Unspecified',
                   'ProductionDateTime': '2018-01-01T04:16:00Z'},
  'GranuleUR': '20180101041600-REMSS-L2P_GHRSST-SSTsubskin-AMSR2-L2B_rt_r29920-v02.0-fv01.0.nc',
  'MetadataSpecification': { 'Name': 'UMM-G',
                             'URL': 'https://cdn.earthdata.nasa.gov/umm/granule/v1.5',
                             'Version': '1.5'},
  'ProviderDates': [{'Date': '2018-01-01T04:16:00Z', 'Type': 'Insert'}],
  'RelatedUrls': [ { 'Type': 'GET DATA',
                     'URL': 's3://podaac-dev-l2ss-samples/20180101041600-REMSS-L2P_GHRSST-SSTsubskin-AMSR

We can see that the data URL here is an AWS S3 URL:
s3://podaac-dev-l2ss-samples/20180101041600-REMSS-L2P_GHRSST-SSTsubskin-AMSR2-L2B_rt_r29920-v02.0-fv01.0.nc

##  Subset of a PO.DAAC Granule

We can now build onto the root URL in order to actually perform a transformation.  The first transformation is a subset of a selected granule.  _At this time, this requires discovering the granule id from CMR_.  That information can then be appended to the root URL and used to call Harmony with the help of the `request` library.

Above we show how to find a granule id for processing.

**Notes:**
  The L2 subsetter current streams the data back to the user, and does not stage data in S3 for redirects. This is functionality we will be adding over time.
  It doesn't work with URS backed files, which is coming in the next few weeks
  it only works on the show dataset, but 
    

In [37]:
import shutil 

granuleConfig = {
    'granule_id' : 'G1226018995-POCUMULUS',
}
#-90%2C-45%2C90%2C45
spatialSubsetConfig = {
    'west' : '-90',
    'south' : '-45',
    'east' : '90',
    'north' : '45'
}
eoss_var_subset_url = eoss_collection_root+'{granule_id}/?bbox={west},{south},{east},{north}'.format(**granuleConfig, **spatialSubsetConfig)

print('Request URL', eoss_var_subset_url)

with request.urlopen(eoss_var_subset_url) as response, open('temp.nc', 'wb') as out_file:
    print('Stream response, not saved to S3:', response.geturl())
    print('Content Size:', response.headers['Content-length'])
    shutil.copyfileobj(response, out_file)
    print("Downloaded request to temp.nc")

Request URL https://harmony.uat.earthdata.nasa.gov/C1225996408-POCUMULUS/eoss/0.1.0/items/G1226018995-POCUMULUS/?bbox=-90,-45,90,45
Stream response, not saved to S3: https://harmony.uat.earthdata.nasa.gov/C1225996408-POCUMULUS/eoss/0.1.0/items/G1226018995-POCUMULUS/?bbox=-90,-45,90,45
Content Size: 6381633
Downloaded request to temp.nc
